In [ ]:
import config
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
from IPython.display import display
import json
import datetime
import math

from sklearn import metrics
from sklearn.metrics import roc_curve, auc, roc_auc_score, precision_recall_fscore_support, f1_score, cohen_kappa_score, average_precision_score

In [ ]:
STAGE = 'ss'
MOD_PREFIX = "mod_smallimg3"
NEPOCH = 'latest'


DATAPATH = config.OUTPUT_PATH
base_dir = config.RAW_DATA_PATH
img_path = config.SCHULTHESS_DATAPATH
proc_dir = config.PROC_DATA_PATH

folder = "2025-11-19_hdbscan"
run = "run10"  #"run175"

anomalyscore_metric = "centre_mean"
cluster_col = "cluster_label"

In [ ]:
today = datetime.date.today()
folder_date = folder.split('_')[0]

filepath = os.path.join(proc_dir, folder, "pipeline", run)
save_path = os.path.join(filepath, "img")
os.makedirs(save_path, exist_ok=True)


hdbscan_df = pd.read_csv(os.path.join(filepath, f'pipeline_{run}_umap_hdbscan_scaled.csv'))
kl = pd.read_csv(os.path.join(base_dir,  "brul_knee_annotations.csv"))
kl2 = pd.read_csv(os.path.join(base_dir, "rosand1_knee_annotations.csv"))
hdbscan_df = hdbscan_df.merge(kl, left_on = 'id', right_on='name', how='left', validate='one_to_one')
mri = pd.read_csv(os.path.join(base_dir, '2025-09-25_mrismall.csv'))

with open(os.path.join(filepath, f'pipeline_{run}_umap_hdbscan_scaled_model_info.json')) as f:
    model_info= json.load(f)

In [ ]:
hdbscan_df = hdbscan_df.merge(kl2, left_on = 'id', right_on='name', how='left', validate='one_to_one', suffixes=('', '2'))

In [ ]:
try:
    hdbscan_df.drop(columns=['Unnamed: 0'], inplace=True)
except:
    pass

In [ ]:
hdbscan_df.columns
## need to add mri data

In [ ]:
df = pd.read_csv(os.path.join(DATAPATH, 'outputs', 'dfs', 'ss', 'mod_smallimg3_ss_aggregated_scores.csv'))

In [ ]:
df['id_temp'] = df['id'].apply(lambda x: x.split('/')[-1])

In [ ]:
df['id'] = df['id_temp'].apply(lambda x: x.split('.')[0])

In [ ]:
df.drop(columns=['id_temp'], inplace=True)

In [ ]:
hdbscan_df.head()

In [ ]:
hdbscan_df[hdbscan_df['cluster_label'].isna()]

In [ ]:
df_hdbscan = df.merge(hdbscan_df[['id', 'cluster_label']], on='id', how='left', validate='one_to_one')

In [ ]:
df_hdbscan[df_hdbscan['cluster_label'].isna()]

In [ ]:
print(hdbscan_df.shape)
print(df.shape)
print(df_hdbscan.shape)

In [ ]:
df_hdbscan['cluster_label'] = df_hdbscan['cluster_label'].fillna(-1)


In [ ]:
df_hdbscan = df_hdbscan.merge(kl[['name', 'KL-Score']], left_on='id', right_on='name', how='left', validate='one_to_one')
df_hdbscan = df_hdbscan.merge(kl2[['name', 'KL-Score']], left_on='id', right_on='name', how='left', validate='one_to_one', suffixes=('', '2'))

In [ ]:
# if 'KL-Score'  is na, fill with 'KL-Score2'
df_hdbscan['KL-Score'] = df_hdbscan['KL-Score'].fillna(df_hdbscan['KL-Score2'])
df_hdbscan['KL-Score2'] = df_hdbscan['KL-Score2'].fillna(df_hdbscan['KL-Score'])

In [ ]:
df_hdbscan[df_hdbscan['KL-Score'].isna()]

# Quick KL-comparison

In [ ]:
kl['KL-Score'].value_counts().sort_index()

In [ ]:
kl2['KL-Score'].value_counts().sort_index()

# Quick Cluster Label Overview

In [ ]:
df_hdbscan['cluster_label'].value_counts().sort_index() 

In [ ]:
hdbscan_df['cluster_label'].value_counts().sort_index()

# Comparison KL-Score (AUC/SRC)

In [ ]:
def get_metrics(df, score, label_name = 'KL-Score'):

    res = stats.spearmanr(df[score].tolist(), df[label_name].tolist())

    df['binary_label'] = 0
    df.loc[df[label_name] > 0, 'binary_label'] = 1
    fpr, tpr, thresholds = roc_curve(np.array(df['binary_label']),np.array(df[score]))
    auc = metrics.auc(fpr, tpr)


    df['binary_label'] = 0
    df.loc[df[label_name] > 1, 'binary_label'] = 1

    fpr, tpr, thresholds = roc_curve(np.array(df['binary_label']),np.array(df[score]))
    auc_mid = metrics.auc(fpr, tpr)


    df['binary_label'] = 0
    df.loc[df[label_name] > 2, 'binary_label'] = 1
    fpr, tpr, thresholds = roc_curve(np.array(df['binary_label']),np.array(df[score]))
    auc_mid2 = metrics.auc(fpr, tpr)


    df['binary_label'] = 0
    df.loc[df[label_name] == 4, 'binary_label'] = 1
    fpr, tpr, thresholds = roc_curve(np.array(df['binary_label']),np.array(df[score]))
    auc_sev = metrics.auc(fpr, tpr)



    return res[0], auc, auc_mid, auc_mid2, auc_sev

In [ ]:
hdbscan_df['KL-Score2'] = hdbscan_df['KL-Score2'].fillna(hdbscan_df['KL-Score'])

In [ ]:
hdbscan_df['KL-Score'] = hdbscan_df['KL-Score'].fillna(hdbscan_df['KL-Score2'])

## Compare Cluster Label to KL-Score

In [ ]:
hdbscan_df_dropna = hdbscan_df.dropna(subset=['KL-Score', 'KL-Score2'])

metrics_clkl = get_metrics(hdbscan_df_dropna, 'cluster_label', label_name = 'KL-Score')
metrics_clkl2 = get_metrics(hdbscan_df_dropna, 'cluster_label', label_name = 'KL-Score2')

In [ ]:
print(metrics_clkl)

In [ ]:
print(metrics_clkl2)

'Osteophytes', 'Joint-Space-Narrowing', 'pain'

In [ ]:
def evaluate_all_as(df, as_cols, label_name='KL-Score'):

    results = {}
    for col in as_cols:
        results[col] = get_metrics(df, col, label_name=label_name)

    # Create DataFrame
    results_df = pd.DataFrame(results).T
    results_df.columns = ['spearmanr', 'auc', 'auc_mid', 'auc_mid2', 'auc_sev']

    # Calculate mean and std for each metric
    for metric in ['spearmanr', 'auc', 'auc_mid', 'auc_mid2', 'auc_sev']:
        results_df[f'{metric}_mean'] = results_df[metric].mean()
        results_df[f'{metric}_std'] = results_df[metric].std()

    return results_df


### Compare AS to KL-Score

In [ ]:
as_col = df.iloc[:, 1:-3].columns

# use get_metrics to get src, auc, auc_mid, auc_mid2, auc_sev for all as_col and then calculate mean and std of those values
results_df = evaluate_all_as(df_hdbscan, as_col, label_name='KL-Score')
results_df2 = evaluate_all_as(df_hdbscan, as_col, label_name='KL-Score2')

In [ ]:
def ensemble_results(df):
    print(f'SRC mean: {np.round(df['spearmanr_mean'].iloc[0], 3)} with std {np.round(df['spearmanr_std'].iloc[0], 3)}')
    print(f'AUC mean: {np.round((df['auc_mean'].iloc[0])*100, 1)} with std {np.round((df['auc_std'].iloc[0])*100, 1)}')
    print(f'AUC Mid mean: {np.round((df['auc_mid_mean'].iloc[0])*100, 1)} with std {np.round((df['auc_mid_std'].iloc[0])*100, 1)}')
    print(f'AUC Mid2 mean: {np.round((df['auc_mid2_mean'].iloc[0])*100, 1)} with std {np.round((df['auc_mid2_std'].iloc[0])*100, 1)}')
    print(f'AUC Sev mean: {np.round((df['auc_sev_mean'].iloc[0])*100, 1)} with std {np.round((df['auc_sev_std'].iloc[0])*100, 1)}')

In [ ]:
ensemble_results(results_df)

In [ ]:
ensemble_results(results_df2)

### Compare Cluster Label to KL-Score

In [ ]:
df_hdbscan = df_hdbscan.fillna(-1)

In [ ]:
results_df3 = evaluate_all_as(df_hdbscan[df_hdbscan['cluster_label']!=-1], as_col, label_name='cluster_label')

In [ ]:
df_hdbscan[df_hdbscan['cluster_label'].isna()]

In [ ]:
ensemble_results(results_df3)